<h3>Classe Dao que faz a comunicação com o MongoDB</h3>

In [3]:
import pymongo
import json

class WebScrapingDao:

    def connection(self):
        myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/") #mongodb://localhost:27017/
        mydb = myclient["MegaSenna"] # Banco de Dados
        mycol = mydb["historicoJogos"]       # Collection
        return mycol

    def salvarMegaSenna(self, data):        
        client = pymongo.MongoClient("127.0.0.1", 27017) # localhost
        db = client.MegaSenna # Nome do Banco e da Coleção
        #db.tweetData.insert_one(data)
        print(dict(data))
        #db.historicoJogos.insert_many(data)
        db.historicoJogos.insert_one(data)
        
    def findMegaSennaByDataSorteio(self, data_sorteio):
        db_megasenna = WebScrapingDao() # Instância Objeto da Classe
        connected = db_megasenna.connection() # Realiza a conexão com o MongoDB
        return connected.find({"data_sorteio":data_sorteio})

    def listAllMegaSenna(self):
        db_megasenna = WebScrapingDao() # Instância Objeto da Classe
        connected = db_megasenna.connection() # Realiza a conexão com o MongoDB
        return connected.find({})

    def deleteAll(self):
        client = pymongo.MongoClient("localhost", 27017) # 172.0.0.1
        db = client.MegaSenna # Nome do Banco e da Coleção
        db.historicoJogos.delete_many({})
        #db.historicoJogos.delete_one({})

In [4]:
import csv

class Utils:
    # Recebe os registros do Arquivo e Banco para compartação
    def salvarRegistrosNovos(self, dict_file, dict_banco):       
        inter = []
        reader = csv.reader(dict_file) 
        verificar = Utils()                 
        for data_banco in dict_banco:
            #print(str(value_file[0]))             
            for value_file in reader:
                #print(data_banco['dezenas_sorteadas'])                
                if str(value_file[0]) == str(data_banco['data_sorteio']):
                    inter.append(str(value_file[0]))
                    print(str(value_file[0]))

                '''print(str(value_file[0]))
                print("DB: "+str(data_banco['data_sorteio']) + " == CSV: "+str(value_file[0]))
                if str(value_file[0]) == str(data_banco['data_sorteio']):
                    print("EXISTE")
                else:
                    print("NÃO EXISTE")'''
        return inter

    def intersecao(self, dict_file, dict_banco):
        inter = []
        if dict_file == dict_banco:
            inter.append(dict_file)

        '''for x in dict_file:
            for y in dict_banco:
                if x != y:
                    inter.append(x)'''
        return inter

        '''
        reader = csv.reader(dict_file)               
        for row in reader:
            print(str(row[0]))
       
        for data_banco in dict_banco:
            print(data_banco['data_sorteio'])'''

In [7]:
# Importanto as bibliotecas BeautifulSoup e requests nas duas primeiras linhas
from bs4 import BeautifulSoup 
from random import shuffle
#from dao.WebScrapingDao import WebScrapingDao # Importação da classe Dao com MongoDB
import requests
import os.path
import csv
import json # Importação do Módulo JSON para criação do arquivo

# BLOCO DE INSTÂNCIA DO MONGODB    
megaSenna = WebScrapingDao()

# pegando todo o conteúdo de um requisição get na url 
html = requests.get("https://www.resultadosmegasena.com.br/resultados-anteriores").content
soup = BeautifulSoup(html, 'html.parser')
#print(soup.prettify()) # Imprime o html da página

data = []
jogos = {}
table = soup.find('table')
table_body = table.find('tbody')

rows = table_body.find_all('tr', class_="rstable_td")
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values

# ---------------- BLOCO DE ADIÇÃO AO DICIONÁRIO -------------------
for index in range(len(data)):
    # ---------------- BLOCO DE CARREGAMENTO DO ARRAY FORMATADO ---------------
    mesa_senna = {}
    dezenas = []

    mesa_senna['data_sorteio']=data[index][0]
    x = data[index][1].replace('\n', ' ').replace('\t', '') # Retiro os \n e \t do retorno no lugar da quebra adiciono espaço
    value = x.split(' ') # quebro os valores onde tem espaço
    mesa_senna['cd_sorteio']=value[0]
    mesa_senna['ganhadores']=value[2]
    mesa_senna['premio']=value[4].replace('R$','').replace('.','').replace(',','.') # Retiro o símbolo R$ do valor da moeda

    dezenas_sort = data[index][2].split(' ') # Quebro os valores sorteados

    for d in range(len(dezenas_sort)): # listo todos os valores
        #print(int(dezenas_sort[d]))
        dezenas.insert(d,int(dezenas_sort[d])) # Insere elementos a List convertidas em INT

    mesa_senna['dezenas_sorteadas']=dezenas

    jogos[index] = mesa_senna
    # ----------------------------------------------------------------
# ------------------------------------------------------------------

# Verifica se o Arquivo existe
if(os.path.exists('data/megasenna_historico.json')):
    print('Existe arquivo')

    with open('data/megasenna.csv', 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            #print(row)
            #print(row[4]) # jogos
            #print(str(row[0]))
            if(str(row[0]) == '29/01/2020'):
                print('ENCOTRADO')
                with open('data/megasenna.csv', 'a', newline='') as csvfile:
                    fieldnames = ['data_sorteio', 'cd_sorteio', 'ganhadores', 'premio', 'dezenas_sorteadas']
                    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                    #writer.writeheader()                    
                    print(jogos[0])   
                    writer.writerow(jogos[0]) 

else:
    print('Não existe arquivo')

    with open('data/megasenna.csv', 'w', newline='') as csvfile:
        fieldnames = ['data_sorteio', 'cd_sorteio', 'ganhadores', 'premio', 'dezenas_sorteadas']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for i in range(len(jogos.keys())):
            print(jogos[i])   
            writer.writerow(jogos[i]) 
            #writer.writerow({'first_name': 'Baked', 'last_name': 'Beans'})



print(mesa_senna)

Não existe arquivo
{'data_sorteio': '30/09/2020', 'cd_sorteio': '2304', 'ganhadores': '0', 'premio': '78511094.00', 'dezenas_sorteadas': [12, 21, 29, 54, 56, 57]}
{'data_sorteio': '26/09/2020', 'cd_sorteio': '2303', 'ganhadores': '0', 'premio': '50738496.00', 'dezenas_sorteadas': [3, 7, 17, 20, 48, 50]}
{'data_sorteio': '23/09/2020', 'cd_sorteio': '2302', 'ganhadores': '0', 'premio': '43030587.00', 'dezenas_sorteadas': [18, 22, 25, 27, 43, 44]}
{'data_sorteio': '19/09/2020', 'cd_sorteio': '2301', 'ganhadores': '0', 'premio': '36675402.00', 'dezenas_sorteadas': [17, 18, 35, 36, 47, 52]}
{'data_sorteio': '17/09/2020', 'cd_sorteio': '2300', 'ganhadores': '0', 'premio': '31289910.00', 'dezenas_sorteadas': [9, 21, 37, 39, 43, 54]}
{'data_sorteio': '15/09/2020', 'cd_sorteio': '2299', 'ganhadores': '0', 'premio': '27154067.00', 'dezenas_sorteadas': [2, 3, 19, 40, 44, 60]}
{'data_sorteio': '12/09/2020', 'cd_sorteio': '2298', 'ganhadores': '0', 'premio': '6032170.00', 'dezenas_sorteadas': [13, 